## Introduction

## 1. Models:
Models represent simplified or abstract descriptions of a process by which data are generated. Models in pyro are expressed as stochastic functions which implies that models can be composed, reused, imported, and serialized just like regular Python callables.

## 1.2 Distributions

An important class of models (stochastic functions) used explicitly to compute the probability of the outputs given the inputs.Distributions in pyro are implemented in [**pyro.distributions**](http://docs.pyro.ai/distributions.html) which is GPU-accelerated multivariate probability distributions. 

**Example 1**: Let define the unit normal distribution $\mathcal{N}(0,1)$ and draw  sample $x$ and  compute the log probability according to the distribution.

In [15]:
# import some dependencies
import torch
from torch.autograd import Variable

import pyro
import pyro.distributions as dist
import numpy as np

torch.set_printoptions(precision=3)

In [26]:
mu = Variable(torch.zeros(1))   # mean zero
sigma = Variable(torch.ones(1)) # unit variance
x=dist.normal(mu, sigma) 
print("Sample of normal distribution: ", x.data[0])

Sample of normal distribution:  0.10484722256660461


In [17]:
#To compute the log probability according to the distribution
log_p_x = dist.normal.log_pdf(x, mu, sigma)
print("The log probability of {0} is: {1} ".format(x.data[0], log_p_x.data[0]))

The log probability of 0.5654299259185791 is: -1.078794002532959 


**Example 2**: Let define the bernoulli distribution $\mathcal{B}(0,1)$ and draw  sample $x$ and  compute the log probability according to the distribution.

In [11]:
prob = Variable(torch.Tensor([0.6]))
x = dist.bernoulli(prob)
print("Sample of normal distribution: ", x.data[0])

Sample of normal distribution:  1.0


## 1.3 Pyro Sample

Sample  returns a named sample from the unit normal distribution. Pyro’s backend uses these names to uniquely identify sample statements and change their behavior at runtime depending on how the enclosing stochastic function is being used.

**syntax: x = pyro.sample("my_sample", fn, arg1, arg2)**

In [12]:
# Example
x = pyro.sample("x_sample", dist.normal, mu, sigma)
print(x.data[0])

0.04278268665075302


## 1.4 Simple model

Suppose we have a bunch of data with power consumption of a TV with three states OFF, IDLE or ON. We want to reason about how power consumption of TV interacts with whether it was OFF, IDLE or ON. The probability of TV to be in ON state is 0.3 and that of OFF state is 0.7. A simple stochastic function (Model) that does that is given by:

In [46]:
def TV_model():
    on_state = pyro.sample('ON', dist.bernoulli, Variable(torch.Tensor([0.3])))
    on_state = 'ON' if on_state.data[0]==1.0 else 'OFF'
    
    mean_power = {'ON':[80], 'OFF': [0]}[on_state]
    sigma_power = {'ON':[5], 'OFF': [10]}[on_state]
    power = pyro.sample('power', dist.normal,Variable(torch.Tensor(mean_power)), Variable(torch.Tensor(sigma_power)))
                                                      
    return on_state, power.data[0]                                                  

In [47]:
for _ in range(3):
    print(TV_model())

('ON', 84.7545166015625)
('ON', 77.65508270263672)
('OFF', -5.8325934410095215)


We can also build a billing model on top of the above defined model 

In [50]:
def TV_billing():
    on_state, power = TV_model()
    expected_cost = [30] if on_state == 'ON' and power > 80.0 else [5]
    cost = pyro.sample('COST', dist.normal,
                            Variable(torch.Tensor(expected_cost)),
                            Variable(torch.Tensor([10.0])))
    return on_state, cost.data[0]

In [54]:
for _ in range(3):
    print(TV_billing())

('OFF', 1.3554277420043945)
('OFF', -1.06915283203125)
('OFF', -1.44720458984375)


## 2. Inference: Marginal Distribution

Suppose a model generate a joint probability distribution $p(y,z|x)$ over latent variable $z$ and return $y$. Then the question is how to compute the marginal probability of an output $p(y|x)$ or draw sample from marginal distribution.

**Inference**  is the problem of constructing this marginal distribution given an arbitrary boolean constraint so that we can perform the above computations.

To motivate the rest of this tutorial, let’s first build a generative model for a simple physical problem so that we can use Pyro’s inference machinery to solve it.

In pyro the marginalization is implemented on **pyro.infer.Marginal**

### Example

Suppose we are trying to figure out how much power an appliance cunsume, but the meter we’re using is unreliable and gives slightly different answers every time we monitor the same appliance. We could try to compensate for this variability by integrating the noisy measurement information with a guess based on some prior knowledge about the appliance, like its material properties. The following model encodes this process:

In [56]:
def monitor(guess):
    # The prior over weight encodes our uncertainty about our guess
    power = pyro.sample("power", dist.normal, guess, Variable(torch.ones(1)))
    # This encodes our belief about the noisiness of the scale:
    # the measurement fluctuates around the true power
    return pyro.sample("measurement", dist.normal, power, Variable(torch.Tensor([0.75])))

## 2.2 Representing Marginal Distributions
Before using our model to estimate  appliance power consumption let’s  analyzing our model’s behavior.Using importance sampling we can simulate the marginal distribution of measurement values we’d expect to see a priori for a given guess.

To do this using marginalization we need to follow the follwing two steps
- collect a number of weighted execution traces of the model.
- collapse those traces into a histogram over possible return values given a particular set of arguments.